In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
import fastai.basics as fai
import fastai.vision as fv
from pathlib import Path
import pandas as pd
import random
import numpy as np
from collections import defaultdict
import gc
import torch
import torchvision.models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
import math
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import PIL
from tqdm import tqdm_notebook as tqdm
from tqdm import trange

In [4]:
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""

In [5]:
!head train_photo_to_biz_ids.csv

head: cannot open 'train_photo_to_biz_ids.csv' for reading: No such file or directory
